In [ ]:
# run()

In [ ]:
from agent.base import Agent

import gym
import gym_gvgai

import matplotlib.pyplot as plt

# gym_gvgai.dir

%matplotlib inline
import gym
import gym_gvgai
import matplotlib.pyplot as plt
from IPython import display



def show_state(env, step=0, name="", info=""):
    """Fn to visualize the agent playing the game in a notebook
    """
    plt.figure(10)
    plt.clf()
    plt.imshow(env.render(mode="rgb_array"))
    plt.title("{} | Step: {} {}".format(name, step, info))
    plt.axis('off')
    display.clear_output(wait=True)
    display.display(plt.gcf())

In [1]:
import numpy as np
from agent.base import simulate
from generator.levels.base import _initialize
from generator.levels.base import Generator
from generator.levels.CPPNGen import CPPNGen

In [2]:
from utils.TileProcessor import PROCESSOR_NETWORK

In [3]:
model_parameters = filter(lambda p: p.requires_grad, PROCESSOR_NETWORK.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
print(params)

10126


In [4]:
PROCESSOR_NETWORK

Net(
  (conv1): Conv2d(13, 8, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(8, 32, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=96, out_features=48, bias=True)
  (fc2): Linear(in_features=84, out_features=24, bias=True)
  (fc3): Linear(in_features=24, out_features=6, bias=True)
)

In [ ]:
x = np.arange(32).reshape((4, 4, 2))

In [ ]:
x

In [ ]:
x[:,:,0].T

In [ ]:
import os

In [ ]:
x = CPPNGen(_initialize(os.path.join(gym_gvgai.dir + "/envs/games/zelda_v0/zelda_lvl2.txt")))

In [ ]:
x.mutate(1)

In [ ]:
gym_gvgai.dir

In [ ]:
# sorted(os.listdir(os.path.join(gym_gvgai.dir, 'envs/games/zelda_v0/levels')))

In [ ]:
# for f in sorted(os.listdir(os.path.join(gym_gvgai.dir, 'envs/games/zelda_v0/levels'))):
#     os.remove(os.path.join(gym_gvgai.dir, f'envs/games/zelda_v0/levels/{f}'))

# os.rmdir(os.path.join(gym_gvgai.dir, 'envs/games/zelda_v0/levels'))

In [ ]:
from generator.env_gen_wrapper import GridGame

In [ ]:
from agent.base import simulate

In [ ]:
import numpy as np

In [ ]:
from baselines.common.vec_env.dummy_vec_env import DummyVecEnv

In [ ]:
from agent.NNagent import NNagent

In [ ]:
popLim = 2
population = [None]*popLim

for i in range(len(population)):
    population[i] = NNagent(GridGame(game='zelda', 
                                     play_length=1000, 
                                     path=gym_gvgai.dir + '/envs/games/zelda_v0/', 
                                     lvl_name='zelda_lvl0.txt', 
                                     mechanics=['1', '2', '3', '+', 'g', 'w'], # monsters, key, door, wall
                                  )
                         )

In [ ]:
# population[0].fitness(noisy=True, fn=show_state)

In [ ]:
# population[0].env._fit

In [ ]:
def make():
    def _make():
        return GridGame(game='zelda', 
                         play_length=250, 
                         path=gym_gvgai.dir + '/envs/games/zelda_v0/', 
                         lvl_name='zelda_lvl0.txt', 
                         mechanics=['1', '2', '3', '+', 'g', 'w'], # monster, key, door, wall
                               )
    return _make

In [ ]:
x = make()()

In [ ]:
x.env.env.get_action_meanings()

In [ ]:
def move(action):
    a, b, c, d = x.step(action)
    print(b)
    plt.imshow(d['pic'])

In [ ]:
move(2)

In [ ]:
# env = [make(), make()]

In [ ]:
# from baselines.common.vec_env.shmem_vec_env import ShmemVecEnv

In [ ]:
# envs = ShmemVecEnv(env, context='fork')

In [ ]:
# envs.step_async([0, 0])

In [ ]:
# [pipe.poll() for pipe in envs.parent_pipes]

In [ ]:
# if np.all([pipe.poll() for pipe in envs.parent_pipes]):
#     outs = [pipe.recv() for pipe in envs.parent_pipes]

In [ ]:
# self.waiting_step = False
# obs, rews, dones, infos = zip(*outs)
# z = (envs._decode_obses(obs), np.array(rews), np.array(dones), infos)

In [ ]:
# z = envs2.step_wait()



- kill oldest.   

- Write mutate wrapper that lives at top of Pair that will create a new Pair obj  
- Copy the Agent, mutate the env.  

- Flip fiber bits in the Tensor NN input.  

----  
- Interestingly, POET has a default of `max_children=8` for each reporduce step. 

In [ ]:
population

In [ ]:
for epochs in range(2):
    for i in range(len(population)):
        population[i].fitness(noisy=True)
    
    for i in range(len(population)):
        for _ in range(2):
            population.append(population[i].mutate(0.7))
        
    print(f"parents + children = {len(population)} members")
    population = sorted(population, key=lambda x: x.env.id, reverse=True)[:10] 
    # sort by id, kill Pairs with smallest IDs. 
    print(f"new generation has {len(population)} members")
    


In [ ]:
plt.imshow(population[8].env.env.reset())

In [ ]:
population[8].fitness(noisy=True, fn=show_state)

In [ ]:
i = population[8].mutate(1)

In [ ]:
i.fitness(noisy=True, fn=show_state)

In [ ]:
i2 = i.mutate(1)
i3 = i2.mutate(1)
i4 = i3.mutate(1)

In [ ]:
i4.fitness(noisy=True, fn=show_state)

In [ ]:
population[0].env.generator.cleanup()